# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [75]:
import hvplot.pandas
import pandas as pd
import requests


from api_keys import geoapify_key

In [76]:
city_data_df = pd.read_csv("../the_starter_code/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,2.58,44,16,5.07,GS,1696152472
1,1,at taj,24.2049,23.2857,31.37,24,100,7.12,LY,1696152473
2,2,avarua,-21.2078,-159.7750,25.03,94,75,3.60,CK,1696152473
3,3,bredasdorp,-34.5322,20.0403,16.94,52,0,4.95,ZA,1696152375
4,4,olonkinbyen,70.9221,-8.7187,0.82,65,100,2.83,SJ,1696152474


In [77]:
%%capture --no-display

# Configure the map plot
map_plot_one = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

map_plot_one

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [78]:
max_temp_min = 5
max_temp_max = 500

preferred_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= max_temp_min) &
    (city_data_df["Max Temp"] <= max_temp_max)
]
preferred_weather_df = preferred_weather_df.dropna()

samp_data_df = preferred_weather_df.sample(17)
samp_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
251,251,kununurra,-15.7667,128.7333,33.99,38,77,5.14,AU,1696152559
231,231,selfoss,63.9331,-20.9971,8.36,77,100,6.17,IS,1696152552
468,468,ghat,24.9647,10.1728,31.95,22,95,0.61,LY,1696152639
572,572,kaduy,59.2000,37.1500,14.13,71,84,5.49,RU,1696152675
259,259,nabire,-3.3667,135.4833,24.86,90,100,0.81,ID,1696152562
305,305,sangkhla buri,15.1555,98.4536,25.42,93,100,0.58,TH,1696152577
514,514,papao,-17.7500,-149.5500,24.82,83,2,9.68,PF,1696152655
498,498,antofagasta,-23.6500,-70.4000,15.60,84,54,0.79,CL,1696152588
369,369,tadine,-21.5500,167.8833,20.68,70,97,8.20,NC,1696152598
569,569,brisas de zicatela,15.8369,-97.0419,26.29,83,87,2.22,MX,1696152674


In [79]:
chosen = ['City', 'Country', 'Lat','Lng', 'Max Temp',]
new_hotel_df = samp_data_df[chosen].copy()


new_hotel_df.insert(5, 'Hotel Name', "")
new_hotel_df

,City,Country,Lat,Lng,Max Temp,Hotel Name
251,kununurra,AU,-15.7667,128.7333,33.99,
231,selfoss,IS,63.9331,-20.9971,8.36,
468,ghat,LY,24.9647,10.1728,31.95,
572,kaduy,RU,59.2000,37.1500,14.13,
259,nabire,ID,-3.3667,135.4833,24.86,
305,sangkhla buri,TH,15.1555,98.4536,25.42,
514,papao,PF,-17.7500,-149.5500,24.82,
498,antofagasta,CL,-23.6500,-70.4000,15.60,
369,tadine,NC,-21.5500,167.8833,20.68,
569,brisas de zicatela,MX,15.8369,-97.0419,26.29,


In [80]:
radius = 10000
params = {
    "categories":"accommodation.hotel", 
    "apiKey":geoapify_key, 
    "limit":20
    
}

print("looking for nearest hotels")


for index, row in new_hotel_df.iterrows():
    
    city = row["City"]
   
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    


    base_url = "https://api.geoapify.com/v2/places"

    name_address = requests.get(base_url, params=params)

    name_address = name_address.json()
    
   
    try:
        new_hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
    except (KeyError, IndexError):
      
        new_hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
 
    print(f"{new_hotel_df.loc[index, 'City']} - nearest hotel: {new_hotel_df.loc[index, 'Hotel Name']}")


new_hotel_df

looking for nearest hotels
kununurra - nearest hotel: Kununurra Country Club Resort
selfoss - nearest hotel: Miðgarður
ghat - nearest hotel: فندق تاسيلي
kaduy - nearest hotel: Октябрьская
nabire - nearest hotel: Hotel Anggrek
sangkhla buri - nearest hotel: ศรีแดง โฮเต็ล
papao - nearest hotel: Hiti Moana Villa Lodge
antofagasta - nearest hotel: Hotel Colón
tadine - nearest hotel: Hôtel Nengoné
brisas de zicatela - nearest hotel: Casa de Olas
kaohsiung - nearest hotel: 寒軒國際大飯店
netrakona - nearest hotel: Shourov Hotel সৌরভ হোটেল
yatou - nearest hotel: No hotel found
osterburken - nearest hotel: Römerhof
kone - nearest hotel: Pacifik Appartels
semnan - nearest hotel: مرکز آموزشی رفاهی فرهنگیان - خانه معلم
awjilah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Max Temp,Hotel Name
251,kununurra,AU,-15.7667,128.7333,33.99,Kununurra Country Club Resort
231,selfoss,IS,63.9331,-20.9971,8.36,Miðgarður
468,ghat,LY,24.9647,10.1728,31.95,فندق تاسيلي
572,kaduy,RU,59.2000,37.1500,14.13,Октябрьская
259,nabire,ID,-3.3667,135.4833,24.86,Hotel Anggrek
305,sangkhla buri,TH,15.1555,98.4536,25.42,ศรีแดง โฮเต็ล
514,papao,PF,-17.7500,-149.5500,24.82,Hiti Moana Villa Lodge
498,antofagasta,CL,-23.6500,-70.4000,15.60,Hotel Colón
369,tadine,NC,-21.5500,167.8833,20.68,Hôtel Nengoné
569,brisas de zicatela,MX,15.8369,-97.0419,26.29,Casa de Olas


In [81]:
%%capture --no-display

mptwo = new_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 3,
    color = "City"
)

mptwo

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)